In [1]:
import pandas as pd
from urllib.parse import urlparse
import os
import json
from IPython.display import HTML

In [2]:
def link_maker(val):
    # target _blank to open new window
    return '<a target="_blank" href="{}">{}</a>'.format(val, os.path.basename(urlparse(val).path))

def url_dedupe(val):
    return os.path.basename(urlparse(val).path)

def string_name(val):
    try:
        val = val.split(",")[0].strip()
        val = val + "}"
        val = json.loads(val)
        return dict(val).get("reporting_entity_name")
    except:
        return None

In [5]:
df = pd.read_csv("payer_index.csv")


# select rows where column "string" contains x, keeping both columns
search_term = "Texas"


df = df.dropna(subset=["string"])
# Search DF
df = df[df["string"].str.contains(search_term)]

# Create separate column to act as deduping index using only filename
df["url_index"] = df["url"].apply(url_dedupe)
df = df.drop_duplicates(subset=["url_index"])
df = df.drop("url_index", axis=1)

# Make string prettier
df["string"] = df["string"].apply(string_name) 
# df = df.style.format({"url": link_maker})
# df['url'] = df['url'].apply(lambda x: f'<a href="{x}">{os.path.basename(urlparse(x).path)}</a>')

df = df.sort_values(by=["size"])
HTML(df.to_html(render_links=True, escape=False))
df.to_csv("./_input_csvs/" + search_term + ".csv", index=False)